## CSc 8830: Computer Vision - Assignment 1 


#### With the OAK-D camera, set up your application to show a RGB stream from the mono camera and a depth map stream from the stereo camera simultaneously. Make a note of what is the maximum frame rate and resolution achievable?

In [ ]:
# install depthai & opencv-python
!pip install depthai opencv-python

In [ ]:
import time
import cv2 as cv
import numpy as np
import depthai as dai

# Parameters
# Resolution setting for the RGB camera
RGB_RESOLUTION = dai.ColorCameraProperties.SensorResolution.THE_1080_P

# Resolution setting for the mono cameras  
DEPTH_RESOLUTION = dai.MonoCameraProperties.SensorResolution.THE_400_P

# Dimension for resizing the RGB frame  
DIM = (720, 480)  

# Stereo depth parameters
extended_disp = False  # Flag for enabling extended disparity range
sub_pixelel = False  # Flag for enabling sub-pixel disparity refinement
lr_check = True  # Flag for enabling left-right consistency check

# Create pipeline
pipeline = dai.Pipeline()

# Camera configurations
camera_rgb = pipeline.create(dai.node.ColorCamera)
camera_rgb.setResolution(RGB_RESOLUTION)  # Set the resolution for the RGB camera
xout_rgb = pipeline.createXLinkOut()
xout_rgb.setStreamName("rgb")
camera_rgb.video.link(xout_rgb.input)

left = pipeline.createMonoCamera()
left.setResolution(DEPTH_RESOLUTION)  # Set the resolution for the left mono camera
left.setBoardSocket(dai.CameraBoardSocket.LEFT)

right = pipeline.createMonoCamera()
right.setResolution(DEPTH_RESOLUTION)  # Set the resolution for the right mono camera
right.setBoardSocket(dai.CameraBoardSocket.RIGHT)

depth = pipeline.createStereoDepth()
depth.setExtendedDisparity(extended_disp)  # Set whether to enable extended disparity range
depth.setSubpixel(sub_pixelel)  # Set whether to enable sub-pixel disparity refinement
depth.setLeftRightCheck(lr_check)  # Set whether to enable left-right consistency check
left.out.link(depth.left)
right.out.link(depth.right)

xout = pipeline.createXLinkOut()
xout.setStreamName("disparity")
depth.disparity.link(xout.input)

prev_frame_time = 0  # Variable to store the previous frame time

with dai.Device(pipeline) as device:
    q = device.getOutputQueue(name="disparity", maxSize=4, blocking=False)  # Output queue for disparity frames
    q_rgb = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)  # Output queue for RGB frames
    
    while True:
        new_frame_time = time.time()  # Get current time for FPS calculation
        fps = 1 / (new_frame_time - prev_frame_time) if prev_frame_time else 0  # Calculate frames per second
        prev_frame_time = new_frame_time
        
        # Display RGB frame
        if q_rgb and q_rgb.has():
            in_rgb = q_rgb.get()
            frame_rgb = cv.resize(in_rgb.getCvFrame(), DIM, interpolation=cv.INTER_AREA)
            cv.imshow("rgb", frame_rgb)
        
        # Display depth map
        if q and q.has():
            in_disparity = q.get()  # Blocking call, waits until new data is available
            frame = in_disparity.getFrame()
            if frame is not None:
                # Normalization for better visualization
                frame = (frame * (255 / depth.initialConfig.getMaxDisparity())).astype(np.uint8)
                cv.imshow("disparity", frame)

        print("FPS:", int(fps))  # Print FPS in the console
        
        key = cv.waitKey(1)  # Wait for key press
        if key == ord('q'):  # Break the loop if 'q' is pressed
            cv.destroyAllWindows()  # Close all OpenCV windows
            break


#### 1. Report the calibration matrix for the camera chosen and verify (using an example) the same. 
#### 2. Point the camera to a chessboard pattern or any known set of reference points that lie on the same plane. Capture a series of 10 images by changing the orientation of the camera in each iteration. Select any 1 image, and using the image formation pipeline equation, set up the linear equations in matrix form and solve for intrinsic and extrinsic parameters (extrinsic for that particular orientation). You will need to make measurements of the actual 3D world points, and mark pixel coordinates. Once you compute the Rotation matrix, you also need to compute the angles of rotation along each axis. Choose your order of rotation based on your experimentation setup. 


In [34]:
import time
import glob
import cv2 as cv
import numpy as np
import depthai as dai
from pathlib import Path

'''
    Function to capture 10 images from the specified source (RIGHT or LEFT monochrome camera or RGB camera of OAK-D Lite).
    Images are captured at intervals of delay milliseconds.
    
    Params: 
        src = {'right' || 'left' || 'rgb'} (default: 'right')
        delay = delay in milliseconds (default: 1000)
'''

def captureImages(src='right', delay=1000, num_imgs = 10):
    if src not in ['right', 'left', 'rgb']: 
        print("ENTER CORRECT PARAMS!")
        print("accepted params: {left, right, rgb} ")
        print(f"entered src: {src}")
        return;
    
    # Start defining a pipeline
    pipeline = dai.Pipeline()

    # Define a source - mono (grayscale) camera
    cam = pipeline.createMonoCamera()
    if src == 'right':
        cam.setBoardSocket(dai.CameraBoardSocket.RIGHT)
    elif src == 'left':
        cam.setBoardSocket(dai.CameraBoardSocket.LEFT)
    else:
        print("Invalid source for monochrome camera")
        return

    cam.setResolution(dai.MonoCameraProperties.SensorResolution.THE_480_P)

    # Create output
    xout = pipeline.createXLinkOut()
    xout.setStreamName(src)
    cam.out.link(xout.input)

    # Connect and start the pipeline
    with dai.Device(pipeline, usb2Mode=True) as device:

        # Output queue will be used to get the grayscale frames from the output defined above
        q = device.getOutputQueue(name=src, maxSize=4, blocking=False)

        # Make sure the destination path is present before starting to store the examples
        Path(f"images/{src}").mkdir(parents=True, exist_ok=True)

        for i in range(num_imgs):
            print(f"Capturing image {i+1} from {src} camera...")
            # Blocking call, will wait until a new data has arrived
            inSrc = q.get()  
            # Data is originally represented as a flat 1D array, it needs to be converted into HxW form
            frame = inSrc.getCvFrame()
            # Frame is transformed and ready to be shown
            cv.imshow(src, frame)

            cv.imwrite(f"images/{src}/{int(time.time() * 10000)}.png", frame)
            print(f"Image {i+1} captured from {src} camera")
            cv.waitKey(delay)  

            cv.destroyAllWindows()            


def captureColorImages(delay=1000, num_imgs = 10):
    # Start defining a pipeline
    pipeline = dai.Pipeline()

    # Define a source - color camera
    cam = pipeline.createColorCamera()
    cam.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)

    # Create RGB output
    xout = pipeline.createXLinkOut()
    xout.setStreamName("rgb")
    cam.video.link(xout.input)

    # Connect and start the pipeline
    with dai.Device(pipeline, usb2Mode=True) as device:

        # Output queue will be used to get the color frames from the output defined above
        q = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)

        # Make sure the destination path is present before starting to store the examples
        Path(f"images/rgb").mkdir(parents=True, exist_ok=True)

        for i in range(num_imgs):
            print(f"Capturing image {i+1} from RGB camera...")
            # Blocking call, will wait until a new data has arrived
            inSrc = q.get()  
            # Data is originally represented as a flat 1D array, it needs to be converted into HxW form
            frame = inSrc.getCvFrame()
            # Frame is transformed and ready to be shown
            imS = cv.resize(frame, (960, 540)) # Resize image
            cv.imshow("rgb", imS)   
#             cv.imshow("rgb", frame)

            cv.imwrite(f"images/rgb/{int(time.time() * 10000)}.png", frame)
            print(f"Image {i+1} captured from RGB camera")
            cv.waitKey(delay)  

            cv.destroyAllWindows()


'''
    Function to find corners, calibrate, and store camera matrix and distortion vector from the specified source 
    (RIGHT or LEFT monochrome camera or RGB camera of OAK-D Lite).
    
    Params: 
        images = array of image paths
        src = source file {'right' || 'left' || 'rgb'} 
'''

def caliberate(images, src):
    if src not in ['right', 'left', 'rgb']: 
        print("ENTER CORRECT PARAMS!")
        print("accepted params: {left, right, rgb} ")
        print(f"entered src: {src}")
        return;

    # Termination criteria
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    # Prepare object points
    objp = np.zeros((6*9, 3), np.float32)
    objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2)

    # Arrays to store object points and image points from all the images
    objpoints = []  # 3D point in real world space
    imgpoints = []  # 2D points in image plane

    notFound = []

    for fname in images:
        img = cv.imread(fname)
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

        # Finding chess board corners
        ret, corners = cv.findChessboardCorners(gray, (9, 6), None)

        # If found, add object points, image points (after refining them)
        if ret == True:
            objpoints.append(objp)
            corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
            imgpoints.append(corners)

            # Draw and display the corners
            cv.drawChessboardCorners(img, (9, 6), corners2, ret)
            cv.imshow('img', img)

            # Saving displayed corners for future references
            cv.imwrite(f"{fname.split('.')[0]}_corners.png", img)
            cv.waitKey(1000)
            cv.destroyAllWindows()
        else:
            # If corners not found, store the file name in the notFound list
            notFound.append(fname)
            print(f"Corners not found for {fname}")

    cv.destroyAllWindows()

    # Remove the images whose corners weren't found from the main image list
    for i in notFound:
        images.remove(i)

    # Camera calibration
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

    for fname in images:
        img = cv.imread(fname)
        h,  w = img.shape[:2]
        newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

        # Undistort the image
        dst = cv.undistort(img, mtx, dist, None, newcameramtx)

        # Crop the image
        x, y, w, h = roi
        dst = dst[y:y+h, x:x+w]

        # Save the undistorted image
        cv.imwrite(f"{fname.split('.')[0]}_result.png", dst)

    # Save camera matrix and distortion coefficients for future use
    print("Saving camera matrix...")
    np.savetxt(f"images/{src}/camera_matrix.txt", mtx)
    print("Saving distortion vector...")
    np.savetxt(f"images/{src}/distortion_vector.txt", dist)
    print("Saving rotational vectors...")
    np.savetxt(f"images/{src}/rotation_vectors.txt", np.array(rvecs).reshape(-1, 3))
    print("Saving translation vectors...")
    np.savetxt(f"images/{src}/translation_vectors.txt", np.array(tvecs).reshape(-1, 3))
    print('Done!!')


# Capture images and calibrate cameras
print("Capturing and calibrating cameras...")
# Capture images using the right monochrome camera
captureImages('right')
# Capture images using the left monochrome camera
captureImages('left')
# Capture images using the color camera
captureColorImages()

# Get paths of captured images
right_images = glob.glob('images/right/*.png')
color_images = glob.glob('images/rgb/*.png')
left_images = glob.glob('images/left/*.png')

# Calibrate cameras
print("Calibrating cameras...")
caliberate(right_images, 'right')
caliberate(left_images, 'left')
caliberate(color_images, 'rgb')

Capturing and calibrating cameras...


/var/folders/cv/flgh8s7960bc7q380pyn0c6m0000gn/T/ipykernel_15176/2350202949.py:83: DeprecationWarning: Use constructor taking 'UsbSpeed' instead
  with dai.Device(pipeline, usb2Mode=True) as device:


Capturing image 1 from RGB camera...
Image 1 captured from RGB camera
Capturing image 2 from RGB camera...
Image 2 captured from RGB camera
Capturing image 3 from RGB camera...
Image 3 captured from RGB camera
Capturing image 4 from RGB camera...
Image 4 captured from RGB camera
Capturing image 5 from RGB camera...
Image 5 captured from RGB camera
Capturing image 6 from RGB camera...
Image 6 captured from RGB camera
Capturing image 7 from RGB camera...
Image 7 captured from RGB camera
Capturing image 8 from RGB camera...
Image 8 captured from RGB camera
Capturing image 9 from RGB camera...
Image 9 captured from RGB camera
Capturing image 10 from RGB camera...
Image 10 captured from RGB camera
Calibrating cameras...
Corners not found for images/rgb/17106200177439.png
Corners not found for images/rgb/17106200243587.png
Corners not found for images/rgb/17106200199564.png
Corners not found for images/rgb/17106200210600.png
Saving camera matrix...
Saving distortion vector...
Saving rotation

In [33]:
import cv2
import numpy as np

def load_camera_data(camera_matrix_file, distortion_vector_file):
    camera_matrix = []
    extrinsic_matrix = []
    with open(camera_matrix_file, 'r') as f:
        for line in f:
            camera_matrix.append([float(num) for num in line.split(' ')])
    with open(distortion_vector_file, 'r') as f:
        for line in f:
            extrinsic_matrix.append([float(num) for num in line.split(' ')])
    return np.array(camera_matrix), np.array(extrinsic_matrix)

def undistort_image(img, camera_matrix, extrinsic_matrix):
    h, w = img.shape[:2]
    new_camera_mtx, roi = cv2.getOptimalNewCameraMatrix(camera_matrix, extrinsic_matrix, (w, h), 1, (w, h))
    dst = cv2.undistort(img, camera_matrix, extrinsic_matrix, None, new_camera_mtx)
    x, y, w, h = roi
    dst = dst[y:y+h, x:x+w]
    return dst

def calculate_calibration_error(corners, camera_matrix, obj_dist, known_square_size_mm):
    ret, corners = cv2.findChessboardCorners(corners, (9, 6), None)
    focal_length_pixels = (camera_matrix[0, 0] + camera_matrix[1, 1]) / 2
    if ret:
        square_size_pixels = np.linalg.norm(corners[3] - corners[4])
        square_size_mm = (square_size_pixels / focal_length_pixels) * obj_dist 
        print(f"Size of Square in mm: {square_size_mm}")
        error = abs(square_size_mm - known_square_size_mm)
        return error
    return None

img_to_undistort = cv2.imread('images/left/17104637414935.png')
camera_matrix, extrinsic_matrix = load_camera_data('images/left/camera_matrix.txt', 'images/left/distortion_vector.txt')
undistorted_img = undistort_image(img_to_undistort, camera_matrix, extrinsic_matrix)
gray = cv2.cvtColor(undistorted_img, cv2.COLOR_BGR2GRAY)
known_square_size_mm = 30  # Set the known size of the chessboard square in mm
obj_dist = 200  # Set the distance of the object from the camera in mm
calibration_error = calculate_calibration_error(gray, camera_matrix, obj_dist, known_square_size_mm)
if calibration_error is not None:
    print(f"Calibration Error: {calibration_error} mm")
else:
    print("Chessboard corners not found.")

Size of Square in mm: 32.32487685307927
Calibration Error: 2.3248768530792674 mm


In [40]:
import cv2
import numpy as np
import math

# Load the chessboard image
chessboard_img = cv2.imread('images/left/17104637414935.png')
gray_img = cv2.cvtColor(chessboard_img, cv2.COLOR_BGR2GRAY)

# Find the chessboard corners
ret, corners = cv2.findChessboardCorners(gray_img, (9, 6), None)

if ret:
    # Refine the corner positions
    refined_corners = cv2.cornerSubPix(gray_img, corners, (11, 11), (-1, -1), 
                                       (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
    
    # Draw the corners on the image
    cv2.drawChessboardCorners(chessboard_img, (9, 6), refined_corners, ret)
    cv2.imshow('Corners', chessboard_img)
    cv2.waitKey(5)
    cv2.destroyAllWindows()

# Extract image points for PnP
top_left_corner = refined_corners[0].ravel()     # Top-left corner in the image
top_right_corner = refined_corners[7].ravel()    # Top-right corner in the image
bottom_right_corner = refined_corners[-1].ravel()  # Bottom-right corner in the image
bottom_left_corner = refined_corners[-8].ravel()   # Bottom-left corner in the image

# 2D image points
img_points = np.array([top_left_corner, top_right_corner, bottom_right_corner, bottom_left_corner])

# Define 3D real-world points
obj_points = np.array([[0, 0, 0], [216, 0, 0], [216, 162, 0], [0, 162, 0]], dtype='float32')

# Solve PnP
ret, rvecs, tvecs = cv2.solvePnP(obj_points, img_points, camera_matrix, extrinsic_matrix)

# Convert rotation vectors to rotation matrix
rotation_matrix, _ = cv2.Rodrigues(rvecs)

# Function to convert rotation matrix to euler angles
def rotation_matrix_to_euler_angles(R):
    sy = math.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])

    singular = sy < 1e-6

    if not singular:
        x = math.atan2(R[2, 1], R[2, 2])
        y = math.atan2(-R[2, 0], sy)
        z = math.atan2(R[1, 0], R[0, 0])
    else:
        x = math.atan2(-R[1, 2], R[1, 1])
        y = math.atan2(-R[2, 0], sy)
        z = 0

    return np.array([x, y, z])

# Convert rotation matrix to euler angles
euler_angles = np.degrees(rotation_matrix_to_euler_angles(rotation_matrix))

# Print results
print("Intrinsic Camera Matrix:")
print(camera_matrix, "\n")
print("Extrinsic Rotation Matrix:")
print(rotation_matrix, "\n")
print("Extrinsic Translation Vector:")
print(tvecs, "\n")
print("Rotation Angles across X, Y, Z axes (degrees):")
print(euler_angles, "\n")

Intrinsic Camera Matrix:
[[210.45781727   0.         315.89660879]
 [  0.         210.13076857 238.12538675]
 [  0.           0.           1.        ]] 

Extrinsic Rotation Matrix:
[[-0.99901378 -0.04423422 -0.00384739]
 [ 0.04406837 -0.9983854   0.0358409 ]
 [-0.00542657  0.03563601  0.9993501 ]] 

Extrinsic Translation Vector:
[[130.86988591]
 [ 84.19519903]
 [169.59623503]] 

Rotation Angles across X, Y, Z axes (degrees):
[  2.04225521   0.31092111 177.47421327] 



#### 3. Write a script to find the real world dimensions (e.g. diameter of a ball, side length of a cube) of an object using perspective projection equations. Validate using an experiment where you image an object using your camera from a specific distance (choose any distance but ensure you are able to measure it accurately) between the object and camera. 

In [1]:
import time
import glob
import cv2 as cv
import numpy as np
import depthai as dai
from pathlib import Path

def captureColorImages(delay=1000, num_imgs = 10):
    # Start defining a pipeline
    pipeline = dai.Pipeline()

    # Define a source - color camera
    cam = pipeline.createColorCamera()
    cam.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)

    # Create RGB output
    xout = pipeline.createXLinkOut()
    xout.setStreamName("rgb")
    cam.video.link(xout.input)

    # Connect and start the pipeline
    with dai.Device(pipeline, usb2Mode=True) as device:

        # Output queue will be used to get the color frames from the output defined above
        q = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)

        # Make sure the destination path is present before starting to store the examples
        Path(f"images/object/").mkdir(parents=True, exist_ok=True)

        for i in range(num_imgs):
            print(f"Capturing image {i+1} from RGB camera...")
            # Blocking call, will wait until a new data has arrived
            inSrc = q.get()  
            # Data is originally represented as a flat 1D array, it needs to be converted into HxW form
            frame = inSrc.getCvFrame()
            # Frame is transformed and ready to be shown
            imS = cv.resize(frame, (960, 540)) # Resize image
            cv.imshow("rgb", imS)   
#             cv.imshow("rgb", frame)

            cv.imwrite(f"images/object/circular_object_{i+1}.png", frame)
            print(f"Image {i+1} captured from RGB camera")
            cv.waitKey(delay)  

            cv.destroyAllWindows()

# Capture images using the color camera
captureColorImages()

/var/folders/cv/flgh8s7960bc7q380pyn0c6m0000gn/T/ipykernel_19060/3528505210.py:22: DeprecationWarning: Use constructor taking 'UsbSpeed' instead
  with dai.Device(pipeline, usb2Mode=True) as device:


Capturing image 1 from RGB camera...
Image 1 captured from RGB camera
Capturing image 2 from RGB camera...
Image 2 captured from RGB camera
Capturing image 3 from RGB camera...
Image 3 captured from RGB camera
Capturing image 4 from RGB camera...
Image 4 captured from RGB camera
Capturing image 5 from RGB camera...
Image 5 captured from RGB camera
Capturing image 6 from RGB camera...
Image 6 captured from RGB camera
Capturing image 7 from RGB camera...
Image 7 captured from RGB camera
Capturing image 8 from RGB camera...
Image 8 captured from RGB camera
Capturing image 9 from RGB camera...
Image 9 captured from RGB camera
Capturing image 10 from RGB camera...
Image 10 captured from RGB camera


In [23]:
import cv2
import math
import numpy as np
import depthai as dai
from pathlib import Path

def find_circle_properties(image_path):
    # Read the image
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply GaussianBlur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Use HoughCircles to detect circles
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1, minDist=50, param1=100, param2=30, minRadius=10, maxRadius=250)

    if circles is not None:
        circles = circles[0, :]

        # Assuming there is only one circle, get its properties
        x, y, r = circles[0]

        # Calculate center point
        center = (int(x), int(y))

        # Calculate width and height (diameter)
        width = int(2 * r)
        height = int(2 * r)

        return center, width, height
    else:
        print("No circle detected in the image.")
        return None, None, None

camera_matrix = []
 
with open('images/left/camera_matrix.txt', 'r') as f:
    for line in f :
        camera_matrix.append([float(num) for num in line.split(' ')])

print("Camera Intrinsic Matrix: ", camera_matrix)

img_path = "images/object/circular_object_10.png"
object_dist = 300 # distance of object from camera in mm

# Provide the focal length in pixels (fx)
# Provide the focal length in pixels (fy)
# Provide the distance to the object in millimeters (Z)

fx, fy, Z = camera_matrix[0][0], camera_matrix[1][1], object_dist
(x, y), w, h = find_circle_properties(img_path)
bbox = (x, y, w, h)  # Provide the bounding box coordinates

print(f"fx: {fx}, fy: {fy}, Z: {Z}")
print(f"Center (x, y): {x}, {y}; Width (w): {w}; Height (h): {h}")

Camera Intrinsic Matrix:  [[210.45781726943, 0.0, 315.89660878657725], [0.0, 210.13076857289957, 238.1253867521593], [0.0, 0.0, 1.0]]
fx: 210.45781726943, fy: 210.13076857289957, Z: 300
Center (x, y): 877, 575; Width (w): 486; Height (h): 486


In [24]:
def convert_milli_to_inch(x):
    x = x / 10
    return x / 25.4

def calculate_object_distance(image, bbox, fx, fy, Z):
    # Unpack bounding box coordinates
    x, y, w, h = bbox

    # Calculate image points
    Image_point1x = x
    Image_point1y = y
    Image_point2x = x + w
    Image_point2y = y + h

    # Draw a line between two points
    cv2.line(image, (Image_point1x, Image_point1y-h//2), (Image_point1x, Image_point2y-h//2), (0, 0, 255), 8)

    # Convert image points to real-world coordinates
    Real_point1x = Z * (Image_point1x / fx)
    Real_point1y = Z * (Image_point1y / fy)
    Real_point2x = Z * (Image_point2x / fx)
    Real_point2y = Z * (Image_point2x / fy)

    print("Real World Co-ordinates: ")
    print("\t", Real_point1x)
    print("\t", Real_point1y)
    print("\t", Real_point2x)
    print("\t", Real_point2y)

    # Calculate the distance between two points
    dist = math.sqrt((Real_point2y - Real_point1y) ** 2 + (Real_point2x - Real_point1x) ** 2)

    val = round(convert_milli_to_inch(dist*2)*10, 2)

    # Draw text on the image with the calculated distance
    cv2.putText(image, str(val)+" mm", (Image_point1x - 200, (y + h) // 2 + 5),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imwrite("circilar_object.png", image)

    return val

image = cv2.imread(img_path)
distance = calculate_object_distance(image, bbox, fx, fy, Z)
print("\nDiameter of circular object is: {} mm".format(distance))

Real World Co-ordinates: 
	 1250.1317528309107
	 820.917380027359
	 1942.9071597588722
	 1945.9311112648525

Diameter of circular object is: 104.03 mm
